## Benchmark Lorenz 63 linear and nonlinear filters

In this notebook, we are interested in the sequential inference 



References: 


[1] Evensen, G., 1994. Sequential data assimilation with a nonlinear quasi‐geostrophic model using Monte Carlo methods to forecast error statistics. Journal of Geophysical Research: Oceans, 99(C5), pp.10143-10162.

[2] Asch, M., Bocquet, M. and Nodet, M., 2016. Data assimilation: methods, algorithms, and applications. Society for Industrial and Applied Mathematics.

[3] Bishop, C.H., Etherton, B.J. and Majumdar, S.J., 2001. Adaptive sampling with the ensemble transform Kalman filter. Part I: Theoretical aspects. Monthly weather review, 129(3), pp.420-436. 

[4] Lorenz, E.N., 1963. Deterministic nonperiodic flow. Journal of atmospheric sciences, 20(2), pp.130-141.

[5] Spantini, A., Baptista, R. and Marzouk, Y., 2019. Coupling techniques for nonlinear ensemble filtering. arXiv preprint arXiv:1907.00389.

### The basic steps
To carry out sequential inference in `AdaptiveTransportMap`, we need to carry out a few basic steps:
* **Specify the problem**: Define the state-space model: initial condition, dynamical and observation models (including process and observation noise)
* **Specify the inflation parameters**: Determine the levels of covariance inflation to properly balance the dynamical system and the observations from the truth system
* **Specify the filter**: Choose the ensemble filter to assimilate the observations in the state estimate
* **Perform the sequential inference**: Perform the sequential inference

We will go through all of these here.

In [1]:
using Revise
using LinearAlgebra
using AdaptiveTransportMap
using Statistics
using Distributions
using OrdinaryDiffEq
using JLD

┌ Info: Precompiling AdaptiveTransportMap [bdf749b0-1400-4207-80d3-e689c0e3f03d]
└ @ Base loading.jl:1278
┌ Warning: Type annotations on keyword arguments not currently supported in recipes. Type information has been discarded
└ @ RecipesBase ~/.julia/packages/RecipesBase/92zOw/src/RecipesBase.jl:116
┌ Warning: Type annotations on keyword arguments not currently supported in recipes. Type information has been discarded
└ @ RecipesBase ~/.julia/packages/RecipesBase/92zOw/src/RecipesBase.jl:116
┌ Warning: Type annotations on keyword arguments not currently supported in recipes. Type information has been discarded
└ @ RecipesBase ~/.julia/packages/RecipesBase/92zOw/src/RecipesBase.jl:116
┌ Warning: Type annotations on keyword arguments not currently supported in recipes. Type information has been discarded
└ @ RecipesBase ~/.julia/packages/RecipesBase/92zOw/src/RecipesBase.jl:116
┌ Warning: Type annotations on keyword arguments not currently supported in recipes. Type information has been

In [2]:
using DelimitedFiles

Load some packages to make nice figures

In [3]:
using Plots
default(tickfont = font("CMU Serif", 9), 
        titlefont = font("CMU Serif", 14), 
        guidefont = font("CMU Serif", 12),
        legendfont = font("CMU Serif", 10),
        grid = false)
# Plots.font("sans-serif")
# clibrary(:colorbrewer)
# gr()
pyplot()

using LaTeXStrings
# PyPlot.rc("text", usetex = "true")
# rcParams = PyPlot.PyDict(PyPlot.matplotlib."rcParams")
# rcParams["text.usetex"] = true;
PyPlot.rc("font", family = "CMU Serif")
PyPlot.matplotlib[:rc]("mathtext",fontset="cm")        #computer modern font 
PyPlot.matplotlib[:rc]("font",family="serif",size=12)

using ColorSchemes

The Lorenz-63  model is a three dimensional system that models the atmospheric convection [4]. This system is a classical benchmark problem in data assimilation. The state $\boldsymbol{x} = (x_1, x_2, x_3)$ is governed by the following set of ordinary differential equations:

\begin{equation}
\begin{aligned}
&\frac{\mathrm{d} x_1}{\mathrm{d} t}=\sigma(x_2-x_1)\\
&\frac{\mathrm{d} x_2}{\mathrm{d} t}=x_1(\rho-x_2)-x_2\\
&\frac{\mathrm{d} x_3}{\mathrm{d} t}=x_1 x_2-\beta x_3,
\end{aligned}
\end{equation}

where $\sigma = 10, \beta = 8/3, \rho = 28$. For these values, the system is chaotic and behaves like a strange attractor. We integrate this system of ODEs with time step $\Delta t_{dyn} = 0.05$. The state is fully observed $h(t,\boldsymbol{x}) = \boldsymbol{x}$ with $\Delta t_{obs}=0.1$. The initial distribution $\pi_{\mathsf{X}_0}$ is the standard Gaussian. The process noise is Gaussian with zero mean and covariance $10^{-4}\boldsymbol{I}_3$. The measurement noise has a Gaussian distribution with zero mean and covariance $\theta^2\boldsymbol{I}_3$ where $\theta^2 = 4.0$.

In [6]:
path = "/media/mat/HDD/AdaptiveTransportMap/notebooks/lorenz63/data/"

β_array = collect(0.95:0.01:1.05)
Ne_array = [10, 20, 40, 60, 100, 200, 400, 600]

# model, data = setup_lorenz63(path, Ne_array);

In [7]:
# save(path*"lorenz63_data.jld", "data", data)

In [8]:
data = load(path*"lorenz63_data.jld", "data")

SyntheticData([0.1, 0.2, 0.30000000000000004, 0.4, 0.5, 0.6000000000000001, 0.7000000000000001, 0.8, 0.9, 1.0  …  599.1, 599.2, 599.3000000000001, 599.4, 599.5, 599.6, 599.7, 599.8000000000001, 599.9, 600.0], [-0.2675782448614047, -0.3979026925121759, 1.9730383889430172], [-0.6755019838448029 -2.0920354300269945 … 6.19724400870094 10.844576122188025; -1.423372451862205 -4.474364320558655 … 9.914661066450536 15.89417323599346; 1.546466186483915 1.5234002823196793 … 16.79159166605126 22.90914235875365], [-0.8031615611550582 -2.745807057798384 … 8.538405900933311 8.265516841716813; -1.572408568133937 -2.859491721222728 … 7.226273016798299 15.231247540993673; 1.7142402843278093 1.266698924014553 … 11.47969219981719 24.179453661167436])

In [9]:
Nx = 3
Ny = 3
Δtdyn = 0.05
Δtobs = 0.1

σx = 1e-6
σy = 2.0

ϵx = AdditiveInflation(Nx, zeros(Nx), σx)
ϵy = AdditiveInflation(Ny, zeros(Ny), σy)
tspinup = 200.0
Tspinup = 2000
tmetric = 400.0
Tmetric = 4000
t0 = 0.0
tf = 600.0
Tf = 6000

Tburn = 2000
Tstep = 4000
Tspinup = 2000

f = lorenz63!
h(x, t) = x
# Create a local version of the observation operator
h(x, t, idx) = x[idx]
F = StateSpace(lorenz63!, h)

model = Model(Nx, Ny, Δtdyn, Δtobs, ϵx, ϵy, MvNormal(zeros(Nx), Matrix(1.0*I, Nx, Nx)), Tburn, Tstep, Tspinup, F);

Benchmark for the Lorenz-63 problem with the sequential stochastic EnkF

In [32]:
metric_enkf = benchmark_lorenz63(model, data, path, Ne_array, β_array);

(Ne, β) = (10, 0.95)


Progress: 100%|█████████████████████████████████████████| Time: 0:00:00


  0.479686 seconds (3.51 M allocations: 331.776 MiB, 6.86% gc time)
Ne = 10
Ne 10& β 0.95 RMSE: 1.4063312146337485
(Ne, β) = (10, 0.96)


Progress: 100%|█████████████████████████████████████████| Time: 0:00:00


  0.481681 seconds (3.51 M allocations: 331.776 MiB, 5.96% gc time)
Ne = 10
Ne 10& β 0.96 RMSE: 3.031626178778555
(Ne, β) = (10, 0.97)


...

Progress: 100%|█████████████████████████████████████████| Time: 0:00:24


 24.598230 seconds (189.77 M allocations: 17.803 GiB, 8.97% gc time)
Ne = 600


Progress: 100%|█████████████████████████████████████████| Time: 0:17:12


Ne 600& β 1.05 RMSE: 0.49307306726541833


8-element Array{Any,1}:
 Metrics[Metrics(10, [0.32078591911714366, 0.4441365673287443, 0.6941005779109013, 0.8793263765117602, 0.8633711337936523, 0.6890628532888722, 0.5273699112797657, 0.4969749546922613, 0.73644871637577, 0.8104683288617239  …  0.31779182360591973, 0.23200415551482328, 0.26645162834570074, 0.5011198105086171, 0.2708562624664994, 0.12212871915966879, 0.023131665869033584, 0.32857741765535037, 1.0715025886964598, 1.1436552936346531], 0.5507688120244398, 1.4063312146337485, 2.8479314378856864, [0.222158622569474, 0.25626111181172273, 0.35182739528951196, 0.3663598873158488, 0.27742718283258855, 0.2510984653416751, 0.20495741379437485, 0.21086558444312314, 0.34637713509870655, 0.5363641061791167  …  0.7025508437981807, 0.9874553700838706, 1.266394784290792, 0.90864824841493, 0.7153302649561325, 0.4606300872849913, 0.4142989373004915, 0.575969040925019, 0.46119715365161734, 0.40109212249053455], 0.3935540555412823, 0.46857519896139493, 0.2556010203617164, [0.666666666666

In [33]:
# save(path*"metric_enkf.jld", "metric", metric_enkf)

Benchmark for the Lorenz-63 problem with the sequential stochastic radial map filter

p = 0

In [34]:
p = 0

0

In [35]:
metric_srmf0 = benchmark_srmf_lorenz63(model, data, path, Ne_array, β_array, p);

(Ne, β) = (10, 0.95)


Progress: 100%|█████████████████████████████████████████| Time: 0:00:01


  1.772121 seconds (27.52 M allocations: 1.955 GiB, 11.79% gc time)
Ne = 10
Ne 10& β 0.95 RMSE: 1.377551150274605
(Ne, β) = (10, 0.96)


Progress: 100%|█████████████████████████████████████████| Time: 0:00:01


  1.749795 seconds (27.52 M allocations: 1.955 GiB, 11.57% gc time)
Ne = 10
Ne 10& β 0.96 RMSE: 1.3657938379515508
(Ne, β) = (10, 0.97)


...

Progress: 100%|█████████████████████████████████████████| Time: 0:01:04


 64.600054 seconds (1.36 G allocations: 95.559 GiB, 11.77% gc time)
Ne = 600


Progress: 100%|█████████████████████████████████████████| Time: 0:30:20


Ne 600& β 1.05 RMSE: 0.4874273248270539


8-element Array{Any,1}:
 Metrics[Metrics(10, [0.7178299117743561, 0.9938479094935131, 0.9157202489871709, 1.7712761491118478, 1.7066313005793774, 1.234897416217821, 0.9317857499383253, 0.6183698609083331, 0.7459082799620395, 1.224062120906078  …  0.6952026612202916, 0.9993289304490182, 0.3801055113850766, 0.46615893854686297, 0.4685463233371965, 0.19599471667823726, 0.19814634786104166, 0.6642794850872766, 0.9773959376079814, 0.818387844848636], 0.6979254430045315, 1.377551150274605, 2.384522230620822, [0.47066072782717444, 0.5827128366257848, 0.8457783375466058, 0.6341692687525824, 0.5343483349823486, 0.3969491055277674, 0.31329438635989737, 0.25834251139447184, 0.3398157520655201, 0.6173447950526075  …  0.7329739456388363, 0.9738238879762458, 1.1289563142068058, 0.6024237576502817, 0.43218587527191027, 0.2990092779069591, 0.2812331285358531, 0.25017593025848966, 0.41779927421383056, 0.48791228952588145], 0.6215600936267961, 0.6750599769304513, 0.29442611799893653, [0.6666666666666666

In [36]:
# save(path*"metric_srmf"*string(p)*".jld", "metric", metric_srmf0)

p = 1

In [48]:
p = 1

1

In [49]:
metric_srmf1 = benchmark_srmf_lorenz63(model, data, path, [40, 60, 100, 200, 400, 600], β_array, p);

(Ne, β) = (40, 0.95)


Progress: 100%|█████████████████████████████████████████| Time: 0:00:07


  7.667425 seconds (111.02 M allocations: 8.373 GiB, 9.97% gc time)
Ne = 40
Ne 40& β 0.95 RMSE: 0.5623248141706708
(Ne, β) = (40, 0.96)


Progress: 100%|█████████████████████████████████████████| Time: 0:00:07


  7.624376 seconds (110.84 M allocations: 8.353 GiB, 10.11% gc time)
Ne = 40
Ne 40& β 0.96 RMSE: 0.4917612606791061
(Ne, β) = (40, 0.97)


...

Progress: 100%|█████████████████████████████████████████| Time: 0:01:33


 93.757272 seconds (1.57 G allocations: 119.478 GiB, 10.51% gc time)
Ne = 600


Progress: 100%|█████████████████████████████████████████| Time: 0:43:25


Ne 600& β 1.05 RMSE: 0.40071429670473335


6-element Array{Any,1}:
 Metrics[Metrics(40, [0.3905612757756471, 0.8139867466354611, 1.3485675277914884, 1.179238504308546, 1.2810451372875125, 1.0592461068701606, 0.8241712047236466, 0.7636104859091258, 1.0643925098479265, 1.3200429676141046  …  0.7409837949171625, 1.0558043978762472, 0.039491436501118846, 0.3718360525828808, 0.2803636613153089, 0.09844109601623455, 0.0748611068460333, 0.09119306031146204, 0.0164826950941801, 0.28217569146809274], 0.41200975918766014, 0.5623248141706708, 0.47640502257653544, [0.3232162472758554, 0.4587779524376999, 0.6171741637536119, 0.674170739623323, 0.47794946567687274, 0.39872714787172103, 0.31936985165812865, 0.32723117425169396, 0.4652134482773482, 0.5482367881460274  …  0.8600488519351489, 0.8338530382593696, 0.8212288925837654, 0.7798769915330456, 0.5468483638310196, 0.29088387840087265, 0.1853699657371057, 0.19031021641026138, 0.28587984062752936, 0.410167912644227], 0.5135461378175266, 0.5764487537743291, 0.25687130981061257, [1.0, 1.0, 0.

In [50]:
# save(path*"metric_srmf"*string(p)*".jld", "metric", metric_srmf1)

p = 2

In [53]:
p = 2

2

In [55]:
metric_srmf2 = benchmark_srmf_lorenz63(model, data, path, [60, 100, 200, 400, 600], collect(0.97:0.01:1.04), p);

(Ne, β) = (60, 0.97)


Progress:  86%|███████████████████████████████████▍     |  ETA: 0:00:02

Max number of iterations is reached during the optimization

Progress: 100%|█████████████████████████████████████████| Time: 0:00:11


 11.711517 seconds (164.88 M allocations: 12.801 GiB, 9.58% gc time)
Ne = 60
Ne 60& β 0.97 RMSE: 0.8219850806868789
(Ne, β) = (60, 0.98)


...

103.944243 seconds (1.57 G allocations: 121.143 GiB, 10.35% gc time)
Ne = 600
Ne 600& β 1.03 RMSE: 0.382530228891025
(Ne, β) = (600, 1.04)


Progress: 100%|█████████████████████████████████████████| Time: 0:01:38


 98.217608 seconds (1.57 G allocations: 121.152 GiB, 9.67% gc time)
Ne = 600


Progress: 100%|█████████████████████████████████████████| Time: 0:31:26


Ne 600& β 1.04 RMSE: 0.36482068549549823


In [58]:
# save(path*"metric_srmf"*string(p)*".jld", "metric", metric_srmf2)

Benchmark for the Lorenz-63 problem with the sequential stochastic adaptive radial map filter

p = 0

In [10]:
p = 0

0

In [ ]:
metric_sadaptivermf0 = benchmark_sadaptivermf_lorenz63(model, data, path, Ne_array, β_array, p);

In [135]:
metric_sadaptivermf0 = benchmark_sadaptivermf_lorenz63(model, data, path, Ne_array, β_array, p);

(Ne, β) = (10, 0.95)


Progress: 100%|█████████████████████████████████████████| Time: 0:00:01


  1.759577 seconds (26.87 M allocations: 1.895 GiB, 11.13% gc time)
Ne = 10
Ne 10& β 0.95 RMSE: 1.01304403888479
(Ne, β) = (10, 0.96)


Progress: 100%|█████████████████████████████████████████| Time: 0:00:01


  1.731787 seconds (26.87 M allocations: 1.895 GiB, 11.21% gc time)
Ne = 10
Ne 10& β 0.96 RMSE: 1.0222472148729265
(Ne, β) = (10, 0.97)


...

 71.115583 seconds (1.36 G allocations: 94.655 GiB, 12.64% gc time)
Ne = 600
Ne 600& β 1.04 RMSE: 0.6617293926447255
(Ne, β) = (600, 1.05)


Progress: 100%|█████████████████████████████████████████| Time: 0:01:12


 72.730837 seconds (1.36 G allocations: 94.655 GiB, 12.51% gc time)
Ne = 600


Progress: 100%|█████████████████████████████████████████| Time: 0:33:05


Ne 600& β 1.05 RMSE: 0.6656091636878243


In [136]:
save(path*"metric_sadaptivermf0"*string(p)*".jld", "metric", metric_sadaptivermf0)

p = 1

In [137]:
p = 1

1

In [138]:
metric_sadaptivermf1 = benchmark_sadaptivermf_lorenz63(model, data, path, Ne_array, β_array, p);

(Ne, β) = (10, 0.95)
norm((smf.S[idx1]).L.μ) = 

Progress:   0%|                                         |  ETA: 0:08:46

23.490430307210634
norm((smf.S[idx1]).L.L.diag) = 16.727720244038547
norm((smf.S[idx1]).L.μ) = 25.08483084132446
norm((smf.S[idx1]).L.L.diag) = 1.5416801361231656
norm((smf.S[idx1]).L.μ) = 23.491080717323037
norm((smf.S[idx1]).L.L.diag) = 17.263054314448745
norm((smf.S[idx1]).L.μ) = 27.14569396866107
norm((smf.S[idx1]).L.L.diag) = 1.8973782892214273
norm((smf.S[idx1]).L.μ) = 33.22533540618917
norm((smf.S[idx1]).L.L.diag) = 17.08871499944379
norm((smf.S[idx1]).L.μ) = 29.230770763923474
norm((smf.S[idx1]).L.L.diag) = 3.3092027073396086
norm((smf.S[idx1]).L.μ) = 25.08483084132446
norm((smf.S[idx1]).L.L.diag) = 1.5416801361231656
norm((smf.S[idx1]).L.μ) = 36.29470464157689
norm((smf.S[idx1]).L.L.diag) = 2.747653413851464
norm((smf.S[idx1]).L.μ) = 27.14569396866107
norm((smf.S[idx1]).L.L.diag) = 1.8973782892214273
norm((smf.S[idx1]).L.μ) = 34.74037189010323
norm((smf.S[idx1]).L.L.diag) = 8.49307305410089
norm((smf.S[idx1]).L.μ) = 29.230770763923474
norm((smf.S[idx1]).L.L.diag) = 3.309202707

norm((smf.S[idx1]).L.μ) = 37.092135194293455
norm((smf.S[idx1]).L.L.diag) = 2.9058539427351744
norm((smf.S[idx1]).L.μ) = 28.363513433309226
norm((smf.S[idx1]).L.L.diag) = 2.3246451471556253
norm((smf.S[idx1]).L.μ) = 36.137473324030736
norm((smf.S[idx1]).L.L.diag) = 2.683202072861527
norm((smf.S[idx1]).L.μ) = 28.290571779577135
norm((smf.S[idx1]).L.L.diag) = 2.265382922409448
norm((smf.S[idx1]).L.μ) = 50.722283092407835
norm((smf.S[idx1]).L.L.diag) = 3.4891371150065122
norm((smf.S[idx1]).L.μ) = 38.809111367711004
norm((smf.S[idx1]).L.L.diag) = 2.5863392180652762
norm((smf.S[idx1]).L.μ) = 28.363513433309226
norm((smf.S[idx1]).L.L.diag) = 2.3246451471556253
norm((smf.S[idx1]).L.μ) = 20.748548119475046
norm((smf.S[idx1]).L.L.diag) = 2.544293746681307
norm((smf.S[idx1]).L.μ) = 28.290571779577135
norm((smf.S[idx1]).L.L.diag) = 2.265382922409448
norm((smf.S[idx1]).L.μ) = 20.303182010699487
norm((smf.S[idx1]).L.L.diag) = 1.5754391730662616
norm((smf.S[idx1]).L.μ) = 38.809111367711004
norm((smf

norm((smf.S[idx1]).L.μ) = 20.90764507654617
norm((smf.S[idx1]).L.L.diag) = 2.0689226331698887
norm((smf.S[idx1]).L.μ) = 19.742796093274382
norm((smf.S[idx1]).L.L.diag) = 2.415179801731758
norm((smf.S[idx1]).L.μ) = 20.681998552496918
norm((smf.S[idx1]).L.L.diag) = 2.646425262993103
norm((smf.S[idx1]).L.μ) = 19.4024734046726
norm((smf.S[idx1]).L.L.diag) = 2.4874733055416822
norm((smf.S[idx1]).L.μ) = 28.525846146320244
norm((smf.S[idx1]).L.L.diag) = 2.2588116098280837
norm((smf.S[idx1]).L.μ) = 24.606927090702168
norm((smf.S[idx1]).L.L.diag) = 1.7400748356486788
norm((smf.S[idx1]).L.μ) = 19.742796093274382
norm((smf.S[idx1]).L.L.diag) = 2.415179801731758
norm((smf.S[idx1]).L.μ) = 23.772689229414485
norm((smf.S[idx1]).L.L.diag) = 2.407991682090566
norm((smf.S[idx1]).L.μ) = 19.4024734046726
norm((smf.S[idx1]).L.L.diag) = 2.4874733055416822
norm((smf.S[idx1]).L.μ) = 25.521578774122105
norm((smf.S[idx1]).L.L.diag) = 3.114031752428422
norm((smf.S[idx1]).L.μ) = 24.606927090702168
norm((smf.S[idx

...

norm((smf.S[idx1]).L.μ) = 1.4148526251150173e22
norm((smf.S[idx1]).L.L.diag) = 3.719180049868971e12
norm((smf.S[idx1]).L.μ) = 8.572900117759286e24
norm((smf.S[idx1]).L.L.diag) = 4.4403207728549716e24
norm((smf.S[idx1]).L.μ) = 1.4148480785177301e22
norm((smf.S[idx1]).L.L.diag) = 3.004393367962714e12
norm((smf.S[idx1]).L.μ) = 2.288129199720653e25
norm((smf.S[idx1]).L.L.diag) = 2.217069647538114e23
norm((smf.S[idx1]).L.μ) = 2.007277428818677e22
norm((smf.S[idx1]).L.L.diag) = 4.249252293275621e12
norm((smf.S[idx1]).L.μ) = 2.0246226188996e25
norm((smf.S[idx1]).L.L.diag) = 9.792598596188152e22
norm((smf.S[idx1]).L.μ) = 8.572900117759286e24
norm((smf.S[idx1]).L.L.diag) = 4.4403207728549716e24
norm((smf.S[idx1]).L.μ) = 4.513788805256076e24
norm((smf.S[idx1]).L.L.diag) = 8.106518785816841e19
norm((smf.S[idx1]).L.μ) = 2.288129199720653e25
norm((smf.S[idx1]).L.L.diag) = 2.217069647538114e23
norm((smf.S[idx1]).L.μ) = 6.417670331969245e24
norm((smf.S[idx1]).L.L.diag) = 9.991665959403993e19
norm((sm

┌ Warning: dt <= dtmin. Aborting. There is either an error in your model specification or the true solution is unstable.
└ @ SciMLBase /home/mat/.julia/packages/SciMLBase/XuLdB/src/integrator_interface.jl:345
┌ Warning: dt <= dtmin. Aborting. There is either an error in your model specification or the true solution is unstable.
└ @ SciMLBase /home/mat/.julia/packages/SciMLBase/XuLdB/src/integrator_interface.jl:345
┌ Warning: dt <= dtmin. Aborting. There is either an error in your model specification or the true solution is unstable.
└ @ SciMLBase /home/mat/.julia/packages/SciMLBase/XuLdB/src/integrator_interface.jl:345
┌ Warning: dt <= dtmin. Aborting. There is either an error in your model specification or the true solution is unstable.
└ @ SciMLBase /home/mat/.julia/packages/SciMLBase/XuLdB/src/integrator_interface.jl:345
┌ Warning: Instability detected. Aborting
└ @ SciMLBase /home/mat/.julia/packages/SciMLBase/XuLdB/src/integrator_interface.jl:351
┌ Warning: Instability detected. A

norm((smf.S[idx1]).L.μ) = 4.5057729708792773e21
norm((smf.S[idx1]).L.L.diag) = 1.7260510299182115e13
norm((smf.S[idx1]).L.μ) = 9.818960311523819e24
norm((smf.S[idx1]).L.L.diag) = 8.613400755064122e21
norm((smf.S[idx1]).L.μ) = 6.428739702106173e21
norm((smf.S[idx1]).L.L.diag) = 2.233015461331157e13
norm((smf.S[idx1]).L.μ) = 7.416169548449207e28
norm((smf.S[idx1]).L.L.diag) = 2.992692790086962e20
norm((smf.S[idx1]).L.μ) = 3.040919663997985e20
norm((smf.S[idx1]).L.L.diag) = 2.227098155201833e12
norm((smf.S[idx1]).L.μ) = 1.5609296690559863e28
norm((smf.S[idx1]).L.L.diag) = 4.2734851952676854e18
norm((smf.S[idx1]).L.μ) = 9.818960311523819e24
norm((smf.S[idx1]).L.L.diag) = 8.613400755064122e21
norm((smf.S[idx1]).L.μ) = 1.530195848888571e28
norm((smf.S[idx1]).L.L.diag) = 8.299676132538274e15
norm((smf.S[idx1]).L.μ) = 7.416169548449207e28
norm((smf.S[idx1]).L.L.diag) = 2.992692790086962e20
norm((smf.S[idx1]).L.μ) = 2.1640052573871377e28
norm((smf.S[idx1]).L.L.diag) = 9.941907457813272e15
norm(

┌ Warning: Instability detected. Aborting
└ @ SciMLBase /home/mat/.julia/packages/SciMLBase/XuLdB/src/integrator_interface.jl:351
┌ Warning: Instability detected. Aborting
└ @ SciMLBase /home/mat/.julia/packages/SciMLBase/XuLdB/src/integrator_interface.jl:351
┌ Warning: Instability detected. Aborting
└ @ SciMLBase /home/mat/.julia/packages/SciMLBase/XuLdB/src/integrator_interface.jl:351
┌ Warning: Instability detected. Aborting
└ @ SciMLBase /home/mat/.julia/packages/SciMLBase/XuLdB/src/integrator_interface.jl:351
┌ Warning: Instability detected. Aborting
└ @ SciMLBase /home/mat/.julia/packages/SciMLBase/XuLdB/src/integrator_interface.jl:351
┌ Warning: Instability detected. Aborting
└ @ SciMLBase /home/mat/.julia/packages/SciMLBase/XuLdB/src/integrator_interface.jl:351
┌ Warning: Instability detected. Aborting
└ @ SciMLBase /home/mat/.julia/packages/SciMLBase/XuLdB/src/integrator_interface.jl:351
┌ Warning: Instability detected. Aborting
└ @ SciMLBase /home/mat/.julia/packages/SciMLBas

LoadError: [91mRoots.ConvergenceFailed("Stopped at: xn = 7.626170769989281e13. Too many steps taken. ")[39m

In [139]:
save(path*"metric_sadaptivermf1"*string(p)*".jld", "metric", metric_sadaptivermf1)

LoadError: [91mUndefVarError: metric_sadaptivermf1 not defined[39m